In [131]:
# Read the variable maps back in and work on cleaning up/sorting:
import json

with open('variables-ds2.json', 'r') as fp:
    var_dict = json.load(fp)

In [132]:
print(json.dumps(var_dict, indent=4))

{
    "MANONS": {
        "1": "<= 1 Month",
        "2": ">1 Month & <=6 Months",
        "3": ">6 Months & <= 1 Year",
        "4": "> 1 Year",
        "Sysmiss": "-"
    },
    "MANONSA": {
        "4": "-",
        "5": "-",
        "7": "-",
        "9": "-",
        "10": "-",
        "11": "-",
        "12": "-",
        "13": "-",
        "14": "-",
        "15": "-",
        "16": "-",
        "17": "-",
        "18": "-",
        "19": "-",
        "20": "-",
        "21": "-",
        "22": "-",
        "23": "-",
        "24": "-",
        "25": "-",
        "26": "-",
        "27": "-",
        "28": "-",
        "29": "-",
        "30": "-",
        "31": "-",
        "32": "-",
        "33": "-",
        "34": "-",
        "35": "-",
        "36": "-",
        "37": "-",
        "38": "-",
        "39": "-",
        "40": "-",
        "42": "-",
        "43": "-",
        "44": "-",
        "45": "-",
        "51": "-",
        "54": "-",
        "Sysmiss": "-"
    },
  

In [133]:
import numpy as np

In [134]:
def try_to_classify_variables(variables_dict):
    diagnosis = set(['Absent', 'Present'])
    yes_or_no = set(['Yes', 'No'])
    history = set(['Year', 'Month'])
    
    variable_type = {
        'yes_or_no': [],
        'diagnosis': [],
        'history': [],
        'maybe_numerical': [],
        'unsure': [],
    }
    
    new_variable_dict = {}

    for variable in variables_dict.keys():
        v_dict = variables_dict[variable]
        # remove any dashes from values
        reduced_dict = {int(key):value for key, value in v_dict.items() if value != "-"}
        
        # is the dictionary empty after reduction?
        if not reduced_dict:
            variable_type['maybe_numerical'].append(variable)
        else:
            value_words = {word for value in v_dict.values() for word in value.split()}
            print(variable, value_words)
            if value_words == yes_or_no:
                variable_type['yes_or_no'].append(variable)
            elif not diagnosis.isdisjoint(value_words):
                variable_type['diagnosis'].append(variable)
            elif not history.isdisjoint(value_words):
                variable_type['history'].append(variable)
            else:
                variable_type['unsure'].append(variable)
        
        new_variable_dict[variable] = reduced_dict

    return new_variable_dict, variable_type   
            

In [135]:
new_variable_dict, var_types = try_to_classify_variables(var_dict)

MANONS {'>', '<=', '&', 'Month', '>1', '1', 'Months', '<=6', '>6', 'Year', '-'}
MANREC {'>', '<=', '&', 'Month', '>1', '1', 'Months', '<=6', '>6', 'Year', '-'}
DEPONS {'>', '<=', '&', 'Month', '>1', '1', '>6', '<=6', 'Months', 'Year', '-'}
DEPREC {'>', '<=', '&', 'Month', '>1', '1', 'Months', '<=6', '>6', 'Year', '-'}
DYSONS {'>', '<=', '&', 'Month', '>1', '1', '>6', '<=6', 'Months', 'Year', '-'}
DYSREC {'>', '<=', '&', 'Month', '>1', '1', 'Months', '<=6', '>6', 'Year', '-'}
GADONS {'>', '<=', '&', 'Month', '>1', '1', 'Months', '<=6', '>6', 'Year', '-'}
GADREC {'>', '<=', '&', 'Month', '>1', '1', 'Months', '<=6', '>6', 'Year', '-'}
PTONS {'>', '<=', '&', 'Month', '>1', '1', 'Months', '<=6', '>6', 'Year', '-'}
PTREC {'>', '<=', '&', 'Month', '>1', '1', 'Months', '<=6', '>6', 'Year', '-'}
PDONS {'>', '<=', '&', 'Month', '>1', '1', 'Months', '<=6', '>6', 'Year', '-'}
PDREC {'>', '<=', '&', 'Month', '>1', '1', '>6', '<=6', 'Months', 'Year', '-'}
AGONS {'>', '<=', '&', 'Month', '>1', '1', '

In [136]:
len(new_variable_dict)

218

In [137]:
for key, value in var_types.items():
    print(key, len(value))

yes_or_no 48
diagnosis 93
history 26
maybe_numerical 38
unsure 13


In [138]:
var_types['unsure']

['R4EG',
 'SEX',
 'HHCOMP',
 'EDCAT',
 'OLDAGCAT',
 'SEASON',
 'MARSTAT',
 'REL',
 'RACE',
 'INCOME',
 'AGCAT',
 'INCAT',
 'EMP']

In [139]:
with open('variable-types-ds2.json', 'w') as fp:
    json.dump(var_types, fp)

In [140]:
var_types['maybe_numerical']

['MANONSA',
 'MANRECA',
 'DEPONSA',
 'DEPRECA',
 'DYSONSA',
 'DYSRECA',
 'GADONSA',
 'GADRECA',
 'PTONSA',
 'PTRECA',
 'PDONSA',
 'PDRECA',
 'AGONSA',
 'AGRECA',
 'SIMONSA',
 'SIMRECA',
 'SOCONSA',
 'SOCRECA',
 'ALCDONSA',
 'ALCDRECA',
 'ALCAONSA',
 'ALCARECA',
 'DRGDONSA',
 'DRGDRECA',
 'DRGAONSA',
 'DRGARECA',
 'PTSDONSA',
 'AGE',
 'NAPONSA',
 'PTSDRECA',
 'OLDAGE',
 'EDUC',
 'WKHRS',
 'INCDOL',
 'STR',
 'SECU',
 'HHNUM',
 'PVTRT']

In [141]:
import pandas as pd

In [142]:
df_ds2 = pd.read_csv('06693-0002-Data.tsv', delimiter='\t', na_filter=False, na_values='-')

In [143]:
import pymongo

In [144]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [145]:
# Define database and collection
db = client.ncs1_ds2_db
collection = db.variable_maps

In [146]:
def convert_df_labels (df, types, db_collection):
    
    new_df = df.copy()
    
    for key in types.keys():
        if key != 'maybe_numerical':
            for var_name in types[key]:
                var_map = db_collection.find_one({'Name': var_name})['map']
                if type(df[var_name][0]) != str:
                    new_map = {int(key):value for key,value in var_map.items() if key != 'Sysmiss'}
                    new_df[var_name] = df[var_name].map(new_map)
                else:
                    new_map = {key:value for key,value in var_map.items()}
                    new_df[var_name] = df[var_name].map(new_map, na_action=None)

                #new_df[var_name] = df[var_name].map(new_map)
    
    return new_df

In [147]:
new_df_ds2 = convert_df_labels(df_ds2, var_types, collection)

In [148]:
new_df_ds2.head()

,CASEID,MANONS,MANONSA,MANREC,MANRECA,DEPONS,DEPONSA,DEPREC,DEPRECA,DYSONS,...,OLDP2FWT,P2WTV2,PVT,PVTRT,P2WTV3,EMP,EMPST,EMPW,EMPH,EMPOTH
0,10001,NaN,,NaN,,NaN,,NaN,,NaN,...,,,Yes,0.6016,,"Working, incl. temp. laid off, matern./s",No,Yes,No,No
1,10002,NaN,,NaN,,NaN,,NaN,,NaN,...,,,No,2.1321,,"Working, incl. temp. laid off, matern./s",No,Yes,No,No
2,10003,NaN,,NaN,,NaN,,NaN,,NaN,...,,,No,1.3077,,"Working, incl. temp. laid off, matern./s",No,Yes,No,No
3,10004,NaN,,NaN,,NaN,,NaN,,NaN,...,,,Yes,0.7391,,Homemaker,No,No,Yes,No
4,10005,NaN,,NaN,,NaN,,NaN,,NaN,...,,,No,1.2613,,"Working, incl. temp. laid off, matern./s",No,Yes,No,No


In [149]:
new_df_ds2.to_csv("ncs1-ds2.csv", index=False)

In [150]:
new_df_ds2[['AGE', 'EDCAT', 'EMP', 'MARSTAT', 'RACE', 'REL', 'SEX', 'ALCALT1', 'ALCALT2', 'DRGALT1', 'DRGALT2']].head()

,AGE,EDCAT,EMP,MARSTAT,RACE,REL,SEX,ALCALT1,ALCALT2,DRGALT1,DRGALT2
0,26,12 Years,"Working, incl. temp. laid off, matern./s",Marnev,Hispanic,Catholic,Female,Absent,Absent,Absent,Absent
1,44,16+ Years,"Working, incl. temp. laid off, matern./s",Marswd,Black,Protestant,Female,Absent,Absent,Absent,Absent
2,53,0-11 Years,"Working, incl. temp. laid off, matern./s",Marcoh,Black,Protestant,Female,Absent,Absent,Absent,Absent
3,41,13-15 Years,Homemaker,Marswd,White,Protestant,Male,Absent,Absent,Absent,Absent
4,28,12 Years,"Working, incl. temp. laid off, matern./s",Marcoh,White,Catholic,Male,Absent,Absent,Absent,Absent


In [151]:
demographic_df = new_df_ds2[['CASEID', 'AGE', 'EDCAT', 'EMP', 'MARSTAT', 'RACE', 'REL', 'SEX', 'ALCALT1', 'ALCALT2', 'DRGALT1', 'DRGALT2']]

In [152]:
demographic_df.describe()

,CASEID,AGE
count,8098.000000,8098.000000
mean,19828.982341,33.729563
std,5464.512327,10.435057
min,10001.000000,15.000000
25%,12025.250000,26.000000
50%,21907.500000,33.000000
75%,23931.750000,42.000000
max,25956.000000,61.000000


In [153]:
new_df_ds2.MANONS.value_counts()

> 1 Year                 149
>6 Months & <= 1 Year     10
<= 1 Month                 9
>1 Month & <=6 Months      8
Name: MANONS, dtype: int64

In [154]:
np.where(new_df_ds2.isna().sum() != 0)

(array([ 1,  3,  5,  7,  9, 11, 13, 15, 17, 19, 21, 23, 25, 27, 29, 31, 33,
        35, 41, 44, 47, 50, 51, 56, 81], dtype=int64),)

In [155]:
missing_cols = new_df_ds2.columns[np.where(new_df_ds2.isna().sum() != 0)]

In [157]:
df_ds1 = pd.read_csv("ncs1-ds1.csv")

C:\Users\dadra\anaconda\Anaconda3\envs\PythonWebMongo\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (1577,1613,1633,1634,1637,1640,1643,1646,1649,1652,1655,1684,1685,1686,1694,1696,1697,1699,1700,1701,1702,1703,1704,1705,1706,1707,1708,1709,1710,1711,1712,1713,1714,1715,1716,1717,1718,1719,1720,1721,1722,1723,1724,1725,1726,1727,1728,1729,1730,1731,1732,1733,1734,1735,1736,1737,1738,1739,1740,1741,1742,1743,1744,1745,1746,1747,1748,1749,1750,1751,1752,1753,1754,1755,1756,1757,1758,1759,1760,1761,1762,1763,1764,1765,1766,1767,1768,1769,1770,1771,1772,1773,1774,1775,1776,1777,1778,1779,1780,1781,1782,1783,1784,1785,1787,1788,1789,1790,1791,1792,1793,1794,1795,1796,1797,1798,1799,1800,1801,1802,1803,1804,1805,1806,1807,1808,1809,1810,1811,1812,1813,1814,1815,1816,1817,1818,1819,1820,1821,1822,1823,1824,1825,1826,1827,1828,1829,1830,1831,1832,1833,1834,1835,1836,1837,1838,1839,1840,1841,1842,1843,1844,1845,1846,1847,1848,1849,1850,1851,1852,1853,1854,1855,1

In [161]:
%whos

Variable                    Type             Data/Info
------------------------------------------------------
client                      MongoClient      MongoClient(host=['localh<...>ware=False, connect=True)
col                         str              MANONS
collection                  Collection       Collection(Database(Mongo<...>s2_db'), 'variable_maps')
conn                        str              mongodb://localhost:27017
convert_df_labels           function         <function convert_df_labe<...>ls at 0x000002C388A0A6A8>
db                          Database         Database(MongoClient(host<...>ect=True), 'ncs1_ds2_db')
demographic_df              DataFrame              CASEID  AGE        <...>n[8098 rows x 12 columns]
df_ds1                      DataFrame              CASEID        V2  V<...>8098 rows x 2954 columns]
df_ds2                      DataFrame              CASEID MANONS MANON<...>[8098 rows x 224 columns]
fp                          TextIOWrapper    <_io.TextIOWrap

In [193]:
joined_df = df_ds1.merge(new_df_ds2, on='CASEID', lsuffix='ds1')

TypeError: merge() got an unexpected keyword argument 'lsuffix'

In [194]:
new_df_ds2['EDCAT']

0          12 Years
1         16+ Years
2        0-11 Years
3       13-15 Years
4          12 Years
5         16+ Years
6          12 Years
7         16+ Years
8          12 Years
9          12 Years
10         12 Years
11         12 Years
12       0-11 Years
13        16+ Years
14        16+ Years
15         12 Years
16        16+ Years
17         12 Years
18        16+ Years
19         12 Years
20        16+ Years
21        16+ Years
22        16+ Years
23        16+ Years
24      13-15 Years
25       0-11 Years
26      13-15 Years
27      13-15 Years
28         12 Years
29         12 Years
           ...     
8068     0-11 Years
8069     0-11 Years
8070    13-15 Years
8071      16+ Years
8072      16+ Years
8073       12 Years
8074    13-15 Years
8075     0-11 Years
8076    13-15 Years
8077    13-15 Years
8078       12 Years
8079    13-15 Years
8080     0-11 Years
8081      16+ Years
8082       12 Years
8083      16+ Years
8084    13-15 Years
8085      16+ Years
8086       12 Years


In [195]:
DS2_suicide_factors = ['EDCAT', 'RACE', 'EMP', 'AGE', 'MARSTAT', 'REL', 'SEX', 'DEPREC', 'ALCALT1', 'ALCALT2', 'DRGALT2', 'DRGALT1']
DS1_suicide_factors = ['V5114', 'V5118', 'V5113', 
                      'V5115', 'V5112', 'V4428', 'V4433', 'V6305', 'V6301', 
                      'V5225', 'V6649', 'V6749', 'V5918', 'V6143', 'V6126', 
                      'V6114', 'V101', 'V102', 'V6215']
DS1_response = ['V1144']

interest_vars = DS1_response + DS1_suicide_factors + DS2_suicide_factors
interest_vars

['V1144',
 'V5114',
 'V5118',
 'V5113',
 'V5115',
 'V5112',
 'V4428',
 'V4433',
 'V6305',
 'V6301',
 'V5225',
 'V6649',
 'V6749',
 'V5918',
 'V6143',
 'V6126',
 'V6114',
 'V101',
 'V102',
 'V6215',
 'EDCAT',
 'RACE',
 'EMP',
 'AGE',
 'MARSTAT',
 'REL',
 'SEX',
 'DEPREC',
 'ALCALT1',
 'ALCALT2',
 'DRGALT2',
 'DRGALT1']

In [196]:
for col in interest_vars:
    print(joined_df[col].value_counts())

NO      4201
INAP    3511
YES      379
Name: V1144, dtype: int64
A LITTLE      2579
SOMEWHAT      1548
NOT AT ALL    1340
VERY           391
INAP            67
DK               2
Name: V5114, dtype: int64
NOT AT ALL    2871
A LITTLE      2217
SOMEWHAT       629
VERY           131
INAP            67
DK               3
Name: V5118, dtype: int64
A LITTLE      2828
SOMEWHAT      1342
NOT AT ALL    1294
VERY           404
INAP            67
Name: V5113, dtype: int64
A LITTLE      3056
SOMEWHAT      1287
NOT AT ALL    1273
VERY           249
INAP            67
DK               1
Name: V5115, dtype: int64
A LITTLE      2624
SOMEWHAT      1717
NOT AT ALL    1047
VERY           486
INAP            67
DK               2
Name: V5112, dtype: int64
A LITTLE TRUE      2170
SOMEWHAT TRUE      1397
NOT TRUE AT ALL    1353
VERY TRUE           936
INAP                 53
DK                    1
Name: V4428, dtype: int64
NOT TRUE AT ALL    2725
A LITTLE TRUE      1621
SOMEWHAT TRUE       895
VERY TRUE   

In [198]:
new_df_ds2.AGE

0       26
1       44
2       53
3       41
4       28
5       37
6       54
7       41
8       46
9       52
10      26
11      48
12      48
13      45
14      31
15      47
16      40
17      34
18      31
19      42
20      52
21      50
22      43
23      51
24      46
25      54
26      41
27      34
28      28
29      46
        ..
8068    54
8069    16
8070    35
8071    31
8072    27
8073    20
8074    23
8075    33
8076    27
8077    24
8078    18
8079    22
8080    17
8081    26
8082    29
8083    38
8084    32
8085    27
8086    24
8087    30
8088    29
8089    40
8090    25
8091    34
8092    15
8093    15
8094    33
8095    31
8096    48
8097    27
Name: AGE, Length: 8098, dtype: int64

In [199]:
new_df_ds2['MARSTAT'].value_counts()

Marcoh    4410
Marnev    2435
Marswd    1253
Name: MARSTAT, dtype: int64

In [192]:
type(df_ds1['CASEID'][0])

numpy.int64

In [200]:
new_df_ds2.dtypes

CASEID        int64
MANONS       object
MANONSA      object
MANREC       object
MANRECA      object
DEPONS       object
DEPONSA      object
DEPREC       object
DEPRECA      object
DYSONS       object
DYSONSA      object
DYSREC       object
DYSRECA      object
GADONS       object
GADONSA      object
GADREC       object
GADRECA      object
PTONS        object
PTONSA       object
PTREC        object
PTRECA       object
PDONS        object
PDONSA       object
PDREC        object
PDRECA       object
AGONS        object
AGONSA       object
AGREC        object
AGRECA       object
SIMONS       object
             ...   
R4S          object
R4W          object
SWIN         object
SSPR         object
SSUM         object
SFAL         object
IN0          object
IN70         object
INCDOL        int64
INCPERS     float64
STR           int64
SECU          int64
AGCAT        object
AG15         object
AG25         object
AG35         object
AG45         object
INCAT        object
IN0019       object


In [201]:
df_ds1.dtypes

CASEID       int64
V2           int64
V3           int64
V4           int64
V5           int64
V6           int64
V7           int64
V8           int64
V9           int64
V10          int64
V11          int64
V12          int64
V13         object
V14         object
V15          int64
V101        object
V102        object
V103        object
V104        object
V105         int64
V106        object
V107        object
V108        object
V109        object
V110        object
V111        object
V112        object
V113        object
V114        object
V201        object
            ...   
V7417       object
V7418       object
V7419       object
V7420       object
V7421       object
V7422       object
V7423       object
V7424       object
V7425       object
V7426        int64
V7427       object
V7428       object
V7429       object
V7430       object
V7431       object
V7432       object
V7433       object
V7434       object
V7435       object
V7436       object
V7437       object
V7438       

In [202]:
df_ds1[DS1_suicide_factors].dtypes

V5114    object
V5118    object
V5113    object
V5115    object
V5112    object
V4428    object
V4433    object
V6305    object
V6301    object
V5225    object
V6649    object
V6749    object
V5918    object
V6143    object
V6126    object
V6114    object
V101     object
V102     object
V6215    object
dtype: object

In [205]:
new_df_ds2[DS2_suicide_factors].dtypes

EDCAT      object
RACE       object
EMP        object
AGE         int64
MARSTAT    object
REL        object
SEX        object
DEPREC     object
ALCALT1    object
ALCALT2    object
DRGALT2    object
DRGALT1    object
dtype: object

In [206]:
new_df_ds2['DEPREC'].value_counts()

> 1 Year                 620
<= 1 Month               360
>1 Month & <=6 Months    316
>6 Months & <= 1 Year    160
Name: DEPREC, dtype: int64

In [207]:
joined_df = new_df_ds2.join(df_ds1, on='CASEID', lsuffix='ds2')

In [208]:
joined_df.shape

(8098, 3178)

In [213]:
for col in interest_vars:
    print(joined_df[col].value_counts())

NO      4201
INAP    3511
YES      379
Name: V1144, dtype: int64
A LITTLE      2579
SOMEWHAT      1548
NOT AT ALL    1340
VERY           391
INAP            67
DK               2
Name: V5114, dtype: int64
NOT AT ALL    2871
A LITTLE      2217
SOMEWHAT       629
VERY           131
INAP            67
DK               3
Name: V5118, dtype: int64
A LITTLE      2828
SOMEWHAT      1342
NOT AT ALL    1294
VERY           404
INAP            67
Name: V5113, dtype: int64
A LITTLE      3056
SOMEWHAT      1287
NOT AT ALL    1273
VERY           249
INAP            67
DK               1
Name: V5115, dtype: int64
A LITTLE      2624
SOMEWHAT      1717
NOT AT ALL    1047
VERY           486
INAP            67
DK               2
Name: V5112, dtype: int64
A LITTLE TRUE      2170
SOMEWHAT TRUE      1397
NOT TRUE AT ALL    1353
VERY TRUE           936
INAP                 53
DK                    1
Name: V4428, dtype: int64
NOT TRUE AT ALL    2725
A LITTLE TRUE      1621
SOMEWHAT TRUE       895
VERY TRUE   

In [212]:
joined_df = new_df_ds2.join(df_ds1,lsuffix='ds2')

In [214]:
df_ds1.CASEID.dtypes

dtype('int64')

In [215]:
new_df_ds2.CASEID.dtypes

dtype('int64')

In [216]:
joined_df.shape

(8098, 3178)

In [230]:
sub_df = joined_df[['CASEID'] + interest_vars]

In [231]:
sub_df.head()

,CASEID,V1144,V5114,V5118,V5113,V5115,V5112,V4428,V4433,V6305,...,EMP,AGE,MARSTAT,REL,SEX,DEPREC,ALCALT1,ALCALT2,DRGALT2,DRGALT1
0,10001,INAP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,"Working, incl. temp. laid off, matern./s",26,Marnev,Catholic,Female,NaN,Absent,Absent,Absent,Absent
1,10002,NO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,"Working, incl. temp. laid off, matern./s",44,Marswd,Protestant,Female,NaN,Absent,Absent,Absent,Absent
2,10003,NO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,"Working, incl. temp. laid off, matern./s",53,Marcoh,Protestant,Female,NaN,Absent,Absent,Absent,Absent
3,10004,INAP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Homemaker,41,Marswd,Protestant,Male,NaN,Absent,Absent,Absent,Absent
4,10005,INAP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,"Working, incl. temp. laid off, matern./s",28,Marcoh,Catholic,Male,NaN,Absent,Absent,Absent,Absent


In [241]:
sub_df.columns

Index(['CASEID', 'V1144', 'V5114', 'V5118', 'V5113', 'V5115', 'V5112', 'V4428',
       'V4433', 'V6305', 'V6301', 'V5225', 'V6649', 'V6749', 'V5918', 'V6143',
       'V6126', 'V6114', 'V101', 'V102', 'V6215', 'EDCAT', 'RACE', 'EMP',
       'AGE', 'MARSTAT', 'REL', 'SEX', 'DEPREC', 'ALCALT1', 'ALCALT2',
       'DRGALT2', 'DRGALT1'],
      dtype='object')

In [242]:
for col in interest_vars:
    print(col, " : ", sub_df[col].isna().sum())

V1144  :  7
V5114  :  2171
V5118  :  2180
V5113  :  2163
V5115  :  2165
V5112  :  2155
V4428  :  2188
V4433  :  2242
V6305  :  2164
V6301  :  2145
V5225  :  2175
V6649  :  2161
V6749  :  2160
V5918  :  2158
V6143  :  2211
V6126  :  2148
V6114  :  2144
V101  :  2
V102  :  3
V6215  :  2142
EDCAT  :  0
RACE  :  0
EMP  :  0
AGE  :  0
MARSTAT  :  0
REL  :  0
SEX  :  0
DEPREC  :  6642
ALCALT1  :  0
ALCALT2  :  0
DRGALT2  :  0
DRGALT1  :  0


In [235]:
nona_df = sub_df.dropna()

In [236]:
nona_df.shape

(1345, 33)

In [237]:
for col in interest_vars:
    print(col, " : ", nona_df[col].isna().sum())

V1144  :  0
V5114  :  0
V5118  :  0
V5113  :  0
V5115  :  0
V5112  :  0
V4428  :  0
V4433  :  0
V6305  :  0
V6301  :  0
V5225  :  0
V6649  :  0
V6749  :  0
V5918  :  0
V6143  :  0
V6126  :  0
V6114  :  0
V101  :  0
V102  :  0
V6215  :  0
EDCAT  :  0
RACE  :  0
EMP  :  0
AGE  :  0
MARSTAT  :  0
REL  :  0
SEX  :  0
DEPREC  :  0
ALCALT1  :  0
ALCALT2  :  0
DRGALT2  :  0
DRGALT1  :  0


In [238]:
nona_df.to_csv("dropna_join.csv", index=False)

In [240]:
for col in interest_vars:
    print(nona_df[col].value_counts())

NO     1116
YES     229
Name: V1144, dtype: int64
A LITTLE      544
SOMEWHAT      456
NOT AT ALL    181
VERY          153
INAP           11
Name: V5114, dtype: int64
A LITTLE      546
NOT AT ALL    451
SOMEWHAT      259
VERY           77
INAP           11
DK              1
Name: V5118, dtype: int64
A LITTLE      570
SOMEWHAT      419
VERY          178
NOT AT ALL    167
INAP           11
Name: V5113, dtype: int64
A LITTLE      668
SOMEWHAT      401
NOT AT ALL    153
VERY          112
INAP           11
Name: V5115, dtype: int64
SOMEWHAT      525
A LITTLE      492
VERY          205
NOT AT ALL    112
INAP           11
Name: V5112, dtype: int64
A LITTLE TRUE      420
SOMEWHAT TRUE      397
VERY TRUE          347
NOT TRUE AT ALL    170
INAP                11
Name: V4428, dtype: int64
NOT TRUE AT ALL    424
A LITTLE TRUE      368
SOMEWHAT TRUE      302
VERY TRUE          240
INAP                11
Name: V4433, dtype: int64
NO      1152
YES      182
INAP      11
Name: V6305, dtype: int64
NO   

In [243]:
nona_df.columns

Index(['CASEID', 'V1144', 'V5114', 'V5118', 'V5113', 'V5115', 'V5112', 'V4428',
       'V4433', 'V6305', 'V6301', 'V5225', 'V6649', 'V6749', 'V5918', 'V6143',
       'V6126', 'V6114', 'V101', 'V102', 'V6215', 'EDCAT', 'RACE', 'EMP',
       'AGE', 'MARSTAT', 'REL', 'SEX', 'DEPREC', 'ALCALT1', 'ALCALT2',
       'DRGALT2', 'DRGALT1'],
      dtype='object')